In [1]:
import pandas as pd #Python data analysis library
import numpy as np #Python scientific computing
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
trainDf = pd.read_csv("C:/Users/shiv/MARKET BASKET ANALYSIS/Data/order_products__train.csv")
orderDf = pd.read_csv("C:/Users/shiv/MARKET BASKET ANALYSIS/Data/orders.csv")
productDf = pd.read_csv("C:/Users/shiv/MARKET BASKET ANALYSIS/Data/products.csv")

In [3]:
priorDf = pd.read_csv("C:/Users/shiv/MARKET BASKET ANALYSIS/Data/order_products__prior.csv")
trainDf = trainDf.append(priorDf,ignore_index = True)

In [4]:
trainDf['reordered'] = 1 

In [5]:
productCountDf = trainDf.groupby("product_id",as_index = False)["order_id"].count()

In [6]:
newproductCountDf=productCountDf.merge(productDf, left_on='product_id', right_on='product_id', how='inner')
newDf = newproductCountDf[['product_id','product_name']]
newDf

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,3,Robust Golden Unsweetened Oolong Tea
2,9,Light Strawberry Blueberry Yogurt
3,10,Sparkling Orange Juice & Prickly Pear Beverage
4,16,Mint Chocolate Flavored Syrup
...,...,...
21922,49679,Famous Chocolate Wafers
21923,49680,All Natural Creamy Caesar Dressing
21924,49683,Cucumber Kirby
21925,49685,En Croute Roast Hazelnut Cranberry


In [7]:
topLev = 100
productCountDf = productCountDf.sort_values("order_id",ascending = False)
topProdFrame = productCountDf.iloc[0:topLev,:]
productId= topProdFrame.loc[:,["product_id"]]


In [8]:
df = trainDf[0:0]
for i in range(0,99):
    pId = productId.iloc[i]['product_id'] 
    stDf = trainDf[trainDf.product_id == pId ]
    df = df.append(stDf,ignore_index = False)

In [9]:
df.head()


,order_id,product_id,add_to_cart_order,reordered
115,226,24852,2,1
156,473,24852,2,1
196,878,24852,2,1
272,1042,24852,1,1
297,1139,24852,1,1


In [10]:
df=df.merge(newDf, left_on='product_id', right_on='product_id', how='inner')
df=df[['order_id','add_to_cart_order','reordered','product_name']]
df

,order_id,add_to_cart_order,reordered,product_name
0,226,2,1,Banana
1,473,2,1,Banana
2,878,2,1,Banana
3,1042,1,1,Banana
4,1139,1,1,Banana
...,...,...,...,...
45426,9974,1,1,Pure Irish Butter
45427,10187,13,1,Pure Irish Butter
45428,10211,1,1,Pure Irish Butter
45429,10311,2,1,Pure Irish Butter


In [11]:
basket = df.groupby(['order_id', 'product_name'])['reordered'].sum().unstack().reset_index().fillna(0).set_index('order_id')

In [12]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1 

In [13]:
basket_sets = basket.applymap(encode_units)

In [14]:
basket_sets.head()

product_name,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Bartlett Pears,Blueberries,Boneless Skinless Chicken Breasts,...,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Whole Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
basket_sets.size

1395009

In [16]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

In [17]:
frequent_itemsets

,support,itemsets
0,0.014193,(100% Raw Coconut Water)
1,0.022851,(100% Whole Wheat Bread)
2,0.017316,(2% Reduced Fat Milk)
3,0.030871,(Apple Honeycrisp Organic)
4,0.032574,(Asparagus)
...,...,...
133,0.010929,"(Organic Strawberries, Organic Blueberries)"
134,0.010290,"(Organic Hass Avocado, Organic Raspberries)"
135,0.014974,"(Organic Hass Avocado, Organic Strawberries)"
136,0.014619,"(Organic Raspberries, Organic Strawberries)"


In [20]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules=rules[['consequents','antecedent support','consequent support','support','confidence','lift']]

In [21]:
rules=rules[ (rules['lift'] >= 2) &
       (rules['confidence'] >= 0.1) ]
rules

,consequents,antecedent support,consequent support,support,confidence,lift
14,(Organic Raspberries),0.165780,0.057980,0.019445,0.117295,2.023007
15,(Bag of Organic Bananas),0.057980,0.165780,0.019445,0.335373,2.023007
44,(Limes),0.074232,0.059329,0.013271,0.178776,3.013321
45,(Large Lemon),0.059329,0.074232,0.013271,0.223684,3.013321
50,(Limes),0.079767,0.059329,0.011071,0.138790,2.339343
51,(Organic Avocado),0.059329,0.079767,0.011071,0.186603,2.339343
61,(Organic Strawberries),0.043574,0.110993,0.010929,0.250814,2.259734
62,(Organic Raspberries),0.081470,0.057980,0.010290,0.126307,2.178441
63,(Organic Hass Avocado),0.057980,0.081470,0.010290,0.177479,2.178441
66,(Organic Strawberries),0.057980,0.110993,0.014619,0.252142,2.271696
